In [1]:
%cd /raid/ruslan_bazhenov/projects/xray/cargoxray/dataset/data

/raid/ruslan_bazhenov/projects/xray/cargoxray/dataset/data


In [2]:
import shutil
from pathlib import Path

import albumentations as alb
import cv2
import numpy as np
import pandas as pd
import torchvision
from PIL import Image
from torchvision import transforms as tf
import torch
import tqdm
import math


In [3]:
images = pd.read_json('images.json.gz',
                      orient='records',
                      typ='frame',
                      compression='gzip')
images = images.set_index('image_id')
images


,filepath,md5,size,width,height,old_filename,old_location
image_id,,,,,,,
0,data/images/00000.tif,3ab70521b3fdcd223949fda05b60f2f0,15229794,4399,1154,327010D01201909060002.tif,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1,data/images/00001.tif,4873919d851f8bcc8421e3dd6a8c7781,14862814,4293,1154,327010D01201909100004.tif,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
2,data/images/00002.tif,b1ddc51d2a1cf08e7d619bb9e4bda828,13287580,3838,1154,327010D01201909100037.tif,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
3,data/images/00003.tif,187a39ed319c8506d959906f3b510abe,13291042,3839,1154,327010D01201909120012.tif,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
4,data/images/00004.tif,26594213171935bb519219fc95ff6b89,13332586,3851,1154,327010D01201909030024.tif,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...,...
15193,data/images/03B59.tif,1b469be64a9c6963a6f714a0a89225c9,8390562,3496,1200,image2704 (576).tif,raw_backup/old/Готовые для передачи 2806/для п...
15194,data/images/03B5A.tif,6332a0f7a1f8effba22169a2217955a8,10257762,4274,1200,image2704 (540).tif,raw_backup/old/Готовые для передачи 2806/для п...
15195,data/images/03B5B.tif,fccb2a0320903b505703c6221af92781,8047362,3353,1200,image2704 (563).tif,raw_backup/old/Готовые для передачи 2806/для п...


In [4]:
annotations = pd.read_json('annotations.json.gz',
                           orient='records',
                           typ='frame',
                           compression='gzip')
annotations = annotations.set_index('bbox_id')
annotations


,image_id,x_min,y_min,width,height,label
bbox_id,,,,,,
0,470,2760,115,416,893,scooter
1,470,846,637,324,177,scooter
2,455,2077,877,1261,210,spare parts
3,455,672,126,154,67,motorbike
4,114,2116,691,1161,337,tools
...,...,...,...,...,...,...
41998,14366,127,73,4064,750,shoes
41999,14387,122,129,2912,630,clothes
42000,14342,131,119,1156,729,clothes


In [5]:
annotations = annotations\
    .reset_index()\
    .merge(images[['old_location', 'width', 'height']],
           on='image_id',
           suffixes=('_bbox', '_image'))\
    .drop_duplicates(['bbox_id'])\
    .set_index('bbox_id')\
    .rename(columns={'old_location': 'filepath'})
annotations


,image_id,x_min,y_min,width_bbox,height_bbox,label,filepath,width_image,height_image
bbox_id,,,,,,,,,
0,470,2760,115,416,893,scooter,raw_backup/new/Нурик1092/Ainur2308/327010D0120...,3836,1154
1,470,846,637,324,177,scooter,raw_backup/new/Нурик1092/Ainur2308/327010D0120...,3836,1154
2,455,2077,877,1261,210,spare parts,raw_backup/new/Нурик1092/Ainur2308/327010D0120...,3844,1154
3,455,672,126,154,67,motorbike,raw_backup/new/Нурик1092/Ainur2308/327010D0120...,3844,1154
4,114,2116,691,1161,337,tools,raw_backup/new/Нурик1092/Ainur2308/327010D0120...,3836,1154
...,...,...,...,...,...,...,...,...,...
41998,14366,127,73,4064,750,shoes,raw_backup/old/clean/2606/image2704 (573).tif,4298,1200
41999,14387,122,129,2912,630,clothes,raw_backup/old/clean/2606/image2704 (575).tif,3143,1200
42000,14342,131,119,1156,729,clothes,raw_backup/old/clean/2606/image2704 (577).tif,3746,1200


In [6]:

annotations = annotations.assign(width=annotations['width_bbox'] / annotations['width_image'],
                   height=annotations['height_bbox'] / annotations['height_image'],
                   x=(annotations['x_min']+annotations['width_bbox'] / 2) / annotations['width_image'],
                   y=(annotations['y_min']+annotations['height_bbox'] / 2) / annotations['height_image'],)\
                       [['label','x', 'y', 'width', 'height', 'filepath']]


In [7]:
annotations

,label,x,y,width,height,filepath
bbox_id,,,,,,
0,scooter,0.773723,0.486568,0.108446,0.773830,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1,scooter,0.262774,0.628683,0.084463,0.153380,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
2,spare parts,0.704344,0.850953,0.328044,0.181976,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
3,motorbike,0.194849,0.138215,0.040062,0.058059,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
4,tools,0.702946,0.744801,0.302659,0.292028,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...
41998,shoes,0.502327,0.373333,0.945556,0.625000,raw_backup/old/clean/2606/image2704 (573).tif
41999,clothes,0.502068,0.370000,0.926503,0.525000,raw_backup/old/clean/2606/image2704 (575).tif
42000,clothes,0.189269,0.402917,0.308596,0.607500,raw_backup/old/clean/2606/image2704 (577).tif


In [8]:
annotations = annotations.replace((
        'brokkoli',
        'clouthes',
        'equpment',
        'Equipment',
        'grapes',
        'motobike',
        'motorcycle',
        'nectarine',
        'pears',
        'scooters',
        'textiles',
        'tomates',
        'Household goods',
        'Lamps',
        'Car wheels',
        'Clothes',
        'Shoes',
        'Spare parts',
        'appliances',
        'car wheels ',
        'carweels',
        'carwheels',
        'cars',
        'equipment '
    ), (
        'broccoli',
        'clothes',
        'equipment',
        'equipment',
        'grape',
        'bike',
        'bike',
        'nectarin',
        'pear',
        'scooter',
        'textile',
        'tomato',
        'household goods',
        'lamps',
        'car wheels',
        'clothes',
        'shoes',
        'spare parts',
        'appliance',
        'car wheels',
        'car wheels',
        'car wheels',
        'car',
        'equipment'
    ))

In [9]:
labels = annotations[['label', 'filepath']]\
    .drop_duplicates()\
    .groupby('label')\
    .count()['filepath']\
    .sort_values(ascending=False)

labels.name = 'labels'
ignored_labels = labels.iloc[20:]
ignored_labels

label
lamps                36
garlic               35
home appliances      33
bags                 31
pineapple            29
                     ..
agriculturaltires     1
leek                  1
lettuce               1
motorbike parts       1
finished products     1
Name: labels, Length: 81, dtype: int64

In [10]:
selected_labels = labels.iloc[:20]
selected_labels = selected_labels.reset_index()['label']
selected_labels.index.name = 'label_id'
selected_labels

label_id
0               shoes
1             textile
2             clothes
3         spare parts
4             fabrics
5                toys
6           equipment
7           tableware
8          auto parts
9               tires
10              tools
11             pepper
12          appliance
13              grape
14            scooter
15         car wheels
16    household goods
17            bicycle
18             tomato
19           nectarin
Name: label, dtype: object

In [11]:

', '.join(f"'{lbl}'" for lbl in selected_labels.to_list())

"'shoes', 'textile', 'clothes', 'spare parts', 'fabrics', 'toys', 'equipment', 'tableware', 'auto parts', 'tires', 'tools', 'pepper', 'appliance', 'grape', 'scooter', 'car wheels', 'household goods', 'bicycle', 'tomato', 'nectarin'"

In [12]:
ignored_annotations = annotations\
    .reset_index()\
    .merge(ignored_labels,
           on='label')\
    .drop(columns=['labels'])\
    .set_index('bbox_id')

ignored_annotations

,label,x,y,width,height,filepath
bbox_id,,,,,,
3,motorbike,0.194849,0.138215,0.040062,0.058059,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
723,motorbike,0.683869,0.398614,0.090909,0.266898,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
724,motorbike,0.797676,0.381716,0.084757,0.215771,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1158,motorbike,0.956380,0.786828,0.083073,0.253033,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1207,motorbike,0.355636,0.785529,0.081651,0.170711,raw_backup/new/Нурик1092/Ainur2308/Nurik2308/3...
...,...,...,...,...,...,...
32860,scateboard,0.427179,0.400817,0.173333,0.642645,raw_backup/old/clean/3103/MIxed222.tif
32861,scateboard,0.928205,0.371100,0.140513,0.547548,raw_backup/old/clean/3103/MIxed222.tif
32853,ball,0.159359,0.113670,0.038718,0.074294,raw_backup/old/clean/3103/MIxed222.tif


In [13]:
ignored_images = ignored_annotations['filepath'].drop_duplicates()
ignored_images

bbox_id
3        raw_backup/new/Нурик1092/Ainur2308/327010D0120...
723      raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1158     raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1207     raw_backup/new/Нурик1092/Ainur2308/Nurik2308/3...
360      raw_backup/new/Нурик1092/Ainur2308/327010D0120...
                               ...                        
32467             raw_backup/old/clean/05/mixed12 (66).tif
32797             raw_backup/old/clean/3103/Mixed (13).tif
32806             raw_backup/old/clean/3103/Mixed (16).tif
33425              raw_backup/old/clean/3003/mixed (3).tif
32852               raw_backup/old/clean/3103/MIxed222.tif
Name: filepath, Length: 474, dtype: object

In [14]:
ignored_annotations = annotations\
    .reset_index()\
    .merge(ignored_images,
           on='filepath')\
    .set_index('bbox_id')
ignored_annotations


,label,x,y,width,height,filepath
bbox_id,,,,,,
2,spare parts,0.704344,0.850953,0.328044,0.181976,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
3,motorbike,0.194849,0.138215,0.040062,0.058059,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
358,tableware,0.289453,0.501733,0.358594,0.795494,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
359,spare parts,0.785677,0.542461,0.286979,0.835355,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
360,bags,0.578906,0.277296,0.128125,0.324090,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...
41647,equipment,0.274267,0.314583,0.057454,0.095833,raw_backup/old/clean/1706/image2704 (302).tif
41648,shoes,0.819474,0.357500,0.153614,0.326667,raw_backup/old/clean/1706/image2704 (302).tif
41649,clothes,0.791503,0.609167,0.358936,0.458333,raw_backup/old/clean/1706/image2704 (302).tif


In [15]:
selected_labels.to_frame().reset_index()

,label_id,label
0,0,shoes
1,1,textile
2,2,clothes
3,3,spare parts
4,4,fabrics
5,5,toys
6,6,equipment
7,7,tableware
8,8,auto parts
9,9,tires


In [16]:
selected_annotations = pd.concat([annotations, ignored_annotations])\
    .drop_duplicates(keep=False)

selected_annotations = selected_annotations.reset_index().merge(selected_labels.to_frame(
).reset_index()).set_index('bbox_id')[['label', 'label_id', 'x', 'y', 'width', 'height', 'filepath']]

selected_annotations


,label,label_id,x,y,width,height,filepath
bbox_id,,,,,,,
0,scooter,14,0.773723,0.486568,0.108446,0.773830,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1,scooter,14,0.262774,0.628683,0.084463,0.153380,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
13,scooter,14,0.749350,0.304593,0.412851,0.449740,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
14,scooter,14,0.520552,0.271664,0.026015,0.084055,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
57,scooter,14,0.353455,0.265165,0.023207,0.292894,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...,...
20342,household goods,16,0.399709,0.479818,0.638566,0.545573,raw_backup/new/готовые для передачи 01072021/1...
20348,household goods,16,0.524876,0.459635,0.856716,0.510417,raw_backup/new/готовые для передачи 01072021/1...
28719,household goods,16,0.563255,0.226667,0.039571,0.250000,raw_backup/old/clean/pictures-1902-3/mixed wit...


In [26]:
print('Number of object instances:',
      len(selected_annotations[['label', 'filepath']]))

print('Number of labels:',
      len(selected_annotations[['label']].drop_duplicates()))

print('Number of images:',
      len(selected_annotations[['filepath']].drop_duplicates()))

display(selected_annotations[['label', 'filepath']]
        .drop_duplicates()
        .groupby('label')
        .count()
        .rename(columns={'filepath': 'count'})
        .sort_values('count', ascending=False))

display(selected_annotations.reset_index()[['label', 'bbox_id']]
        .drop_duplicates()
        .groupby('label')
        .count()
        .rename(columns={'bbox_id': 'count'})
        .sort_values('count', ascending=False))

Number of object instances: 17699
Number of labels: 20
Number of images: 9464


,count
label,
shoes,1971
textile,1499
clothes,1339
spare parts,1267
fabrics,1222
toys,1051
equipment,673
tableware,386
auto parts,351


,label,bbox_id
0,scooter,0
1,scooter,1
2,scooter,13
3,scooter,14
4,scooter,57
...,...,...
17694,household goods,20342
17695,household goods,20348
17696,household goods,28719
17697,household goods,28721


In [48]:
selected_images = selected_annotations['filepath']\
    .drop_duplicates()\
    .reset_index(drop=True)
selected_images


0       raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1       raw_backup/new/Нурик1092/Ainur2308/327010D0120...
2       raw_backup/new/Нурик1092/Ainur2308/327010D0120...
3       raw_backup/new/Нурик1092/Ainur2308/327010D0120...
4       raw_backup/new/Нурик1092/Ainur2308/327010D0120...
                              ...                        
9459    raw_backup/old/clean/240321_json/mixed with te...
9460    raw_backup/old/clean/240321_json/mixed with te...
9461    raw_backup/old/clean/240321_json/mixed with te...
9462          raw_backup/old/clean/240321_json/441333.tif
9463    raw_backup/old/clean/240321_json/441512 mixed.tif
Name: filepath, Length: 9464, dtype: object

In [49]:
selected_images = images[['old_location', 'width', 'height']]\
    .rename(columns={'old_location': 'filepath'})\
    .drop_duplicates('filepath')\
    .merge(selected_images, on='filepath')\
    .set_index('filepath', drop=True)
selected_images

,width,height
filepath,,
raw_backup/new/Нурик1092/Ainur2308/327010D01201909060002.tif,4399,1154
raw_backup/new/Нурик1092/Ainur2308/327010D01201909100004.tif,4293,1154
raw_backup/new/Нурик1092/Ainur2308/327010D01201909100037.tif,3838,1154
raw_backup/new/Нурик1092/Ainur2308/327010D01201909120012.tif,3839,1154
raw_backup/new/Нурик1092/Ainur2308/327010D01201909030024.tif,3851,1154
...,...,...
raw_backup/old/clean/1-551t_json0118/Mixed463.tif,3387,1346
raw_backup/old/clean/1-551t_json0118/Mixed628.tif,3429,1346
raw_backup/old/clean/1-551t_json0118/mixed 435.tif,3898,1346


In [19]:
bbox_params = alb.BboxParams(format='yolo',
                             label_fields=('labels',))

In [20]:
# loaded_images = {}

# for idx, row in tqdm.tqdm(selected_images.reset_index().iterrows(),
#                           desc='Loading images into RAM',
#                           total=len(selected_images)):
#     img = cv2.imread(row['filepath'], 0)
#     img = cv2.resize(img, (int(row['width'] * 1024 / row['height']), 1024))

#     loaded_images[row['filepath']] = img


In [21]:
train_images = selected_images.sample(frac=0.85)
train_images

,width,height
filepath,,
raw_backup/old/clean/1-121/32701RF06202002150047028.tif,1002,768
raw_backup/old/clean/240321_json/mixed with textile (47).tif,5256,1200
raw_backup/new/готовые для передачи 01072021/Снимки 2606/pictures (741).tif,4022,1346
raw_backup/old/clean/13/Mixed684.tif,3899,1346
raw_backup/old/clean/1-242/32701RF06202002191618034.tif,1062,768
...,...,...
raw_backup/new/готовые для передачи 01072021/снимки2921/Готовые снимки/Готовые2/Mix (88).tif,4023,1346
raw_backup/old/clean/240321_json/mixed with textile (120).tif,5373,1200
raw_backup/new/готовые для передачи 01072021/снимки2921/321/Mixed (97).tif,4022,1346


In [22]:
val_images = pd.concat([selected_images, train_images])\
    .reset_index()\
    .drop_duplicates(keep=False)\
    .set_index('filepath', drop=True)
val_images

,width,height
filepath,,
raw_backup/new/Нурик1092/Ainur2308/327010D01201909090025.tif,4389,1154
raw_backup/new/Нурик1092/Ainur2308/327010D01201909120042.tif,3836,1154
raw_backup/new/Нурик1092/Ainur2308/327010D01201909070024.tif,4391,1154
raw_backup/new/Нурик1092/Ainur2308/327010D01201909090049.tif,4387,1154
raw_backup/new/Нурик1092/Ainur2308/327010D01201909060046.tif,4397,1154
...,...,...
raw_backup/old/clean/1-551t_json0118/Mixed607 (1).tif,2349,1346
raw_backup/old/clean/1-551t_json0118/MIxed515 (1).tif,3550,1346
raw_backup/old/clean/1-551t_json0118/MIxed453 (1).tif,3840,1346


In [23]:
train_images.to_json('yolo/train_images.json', orient='records')
val_images.to_json('yolo/val_images.json', orient='records')

In [24]:
train_annotations = selected_annotations\
    .reset_index()\
    .merge(train_images.reset_index()['filepath'], on='filepath')\
    .set_index('bbox_id')
train_annotations


,label,label_id,x,y,width,height,filepath
bbox_id,,,,,,,
0,scooter,14,0.773723,0.486568,0.108446,0.773830,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
1,scooter,14,0.262774,0.628683,0.084463,0.153380,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
13,scooter,14,0.749350,0.304593,0.412851,0.449740,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
14,scooter,14,0.520552,0.271664,0.026015,0.084055,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
12,toys,5,0.357830,0.559792,0.075182,0.590988,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...,...
20011,household goods,16,0.726553,0.485000,0.503187,0.650000,raw_backup/old/clean/240321_json/mixed with te...
20173,household goods,16,0.071697,0.423750,0.134801,0.655833,raw_backup/old/clean/240321_json/mixed with te...
20210,household goods,16,0.709955,0.517500,0.531300,0.575000,raw_backup/old/clean/240321_json/mixed with te...


In [25]:
val_annotations = selected_annotations\
    .reset_index()\
    .merge(val_images.reset_index()['filepath'], on='filepath')\
    .set_index('bbox_id')
val_annotations


,label,label_id,x,y,width,height,filepath
bbox_id,,,,,,,
74,scooter,14,0.722222,0.425043,0.218049,0.555459,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
73,toys,5,0.327986,0.538995,0.276213,0.805893,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
108,scooter,14,0.078488,0.177210,0.027640,0.186308,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
109,tools,10,0.255280,0.357452,0.168970,0.536395,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
285,scooter,14,0.368613,0.270364,0.038582,0.254766,raw_backup/new/Нурик1092/Ainur2308/327010D0120...
...,...,...,...,...,...,...,...
19885,household goods,16,0.196870,0.438750,0.388282,0.642500,raw_backup/old/clean/240321_json/mixed with te...
19886,household goods,16,0.795488,0.472500,0.366448,0.701667,raw_backup/old/clean/240321_json/mixed with te...
19958,household goods,16,0.198529,0.431250,0.395570,0.637500,raw_backup/old/clean/240321_json/mixed with te...


In [26]:
print(len(train_annotations['label'].drop_duplicates()))
print(len(val_annotations['label'].drop_duplicates()))

20
20


In [28]:
loaded_images = {}

for img in tqdm.tqdm(selected_images.index,
                     desc='Loading images into RAM',
                     total=len(selected_images)):
    im = cv2.imread(img, 0)
    im = cv2.resize(im, (int(im.shape[1] * 1024 / im.shape[0]), 1024))
    loaded_images[img] = im


Loading images into RAM:   3%|▎         | 320/9464 [00:07<03:32, 43.00it/s]


KeyboardInterrupt: 

In [ ]:
weights = 1 / train_annotations[['label', 'filepath']]\
    .drop_duplicates()\
    .groupby('label')\
    .count()\
    .rename(columns={'filepath': 'weight'})['weight']\
    .sort_values()

weights


label
tomato             0.045455
nectarin           0.037037
bicycle            0.013333
household goods    0.010309
car wheels         0.009709
scooter            0.008929
grape              0.008197
pepper             0.007874
appliance          0.004651
tools              0.004348
tires              0.003584
auto parts         0.003300
tableware          0.003067
equipment          0.001695
toys               0.001115
fabrics            0.000957
spare parts        0.000918
clothes            0.000890
textile            0.000797
shoes              0.000602
Name: weight, dtype: float64

In [ ]:
weights = train_annotations[['label', 'filepath']]\
    .drop_duplicates()\
    .merge(weights,on='label')\
    .groupby('filepath')\
    .mean()['weight']
weights


filepath
raw_backup/new/0409completed/327010D01201911060024.tif                                            0.004107
raw_backup/new/0409completed/327010D01201911060025.tif                                            0.001307
raw_backup/new/0409completed/327010D01201911060026.tif                                            0.000918
raw_backup/new/0409completed/327010D01201911060028.tif                                            0.001695
raw_backup/new/0409completed/327010D01201911060039.tif                                            0.000804
                                                                                                    ...   
raw_backup/old/Интеллект21/Интеллект21/Женя/Новая папка/391443 mixed.tif                          0.000918
raw_backup/old/Интеллект21/Интеллект21/Женя/Новая папка/Снимки ИДК для проекта ИИ/mixed 67.tif    0.000602
raw_backup/old/Интеллект21/Интеллект21/Женя/Новая папка/Снимки ИДК для проекта ИИ/mixed 70.tif    0.000602
raw_backup/old/Интеллект21/И

In [ ]:
balanced_train_images = weights.sample(n=100000, replace=True, weights=weights).index.to_series()
balanced_train_images

filepath
raw_backup/old/clean/240321_json/mixed with textile (750).tif                          raw_backup/old/clean/240321_json/mixed with te...
raw_backup/old/clean/10/258.tif                                                                          raw_backup/old/clean/10/258.tif
raw_backup/old/clean/391550-4704277/481053.tif                                            raw_backup/old/clean/391550-4704277/481053.tif
raw_backup/old/clean/05/mixed 604.tif                                                              raw_backup/old/clean/05/mixed 604.tif
raw_backup/new/Нурик1092/Нурик2708/327010D01201911250008.tif                           raw_backup/new/Нурик1092/Нурик2708/327010D0120...
                                                                                                             ...                        
raw_backup/new/готовые для передачи 01072021/0107 done/32701RF06202003042051046.tif    raw_backup/new/готовые для передачи 01072021/0...
raw_backup/old/clean/391550-4704

In [ ]:
# # Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
# path: ../datasets/coco128  # dataset root dir
# train: images/train2017  # train images (relative to 'path') 128 images
# val: images/train2017  # val images (relative to 'path') 128 images
# test:  # test images (optional)

# # Classes
# nc: 80  # number of classes
# names: [ 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
#          'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
#          'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
#          'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
#          'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
#          'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
#          'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
#          'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
#          'hair drier', 'toothbrush' ]  # class names

Path('yolo2').mkdir(exist_ok=True, parents=True)
Path('yolo2/train/images').mkdir(exist_ok=True, parents=True)
Path('yolo2/train/labels').mkdir(exist_ok=True, parents=True)
Path('yolo2/val/images').mkdir(exist_ok=True, parents=True)
Path('yolo2/val/labels').mkdir(exist_ok=True, parents=True)

Path('yolo2/dataset.yaml').write_text(
    'train: train/images\n'
    'val: val/images\n'
    '\n'
    'nc: 20\n'
    'names: [ ' + ', '.join(f"'{lbl}'" for lbl in selected_labels.to_list()) + ' ]\n'
    
)

286

In [ ]:
aug = alb.Compose(
    transforms=[

        alb.Affine(scale=1,
                   rotate=3,
                   shear=3,
                   mode=cv2.BORDER_REFLECT101,
                   p=0.5),

        alb.HorizontalFlip(p=0.5),

        alb.Equalize(p=0.5),

        alb.RandomGamma(gamma_limit=(85, 115),
                        p=0.5),

    ],
    bbox_params=bbox_params
)

In [ ]:
alb_aug = alb.Compose([
    alb.LongestMaxSize(1024),
    alb.PadIfNeeded(1024, 1024, border_mode=cv2.BORDER_CONSTANT,value=255)
], bbox_params=bbox_params)


In [ ]:
counter = 0
for idx, img_path in tqdm.tqdm(enumerate(val_images.index),
                               desc='Val images',
                               total=10):
    img = loaded_images[img_path]

    # img = cv2.imread(img_path, 0)
    # img = cv2.resize(img, (int(img.shape[1] * 1024 / img.shape[0]), 1024))

    sel = val_annotations\
        .loc[val_annotations['filepath'] == img_path].copy()

    bboxes = sel[['x', 'y', 'width', 'height']
                 ].to_records(index=False).tolist()
    labels = sel[['label_id']].to_records(index=False).tolist()

    transformed_sample = alb_aug(image=img, bboxes=bboxes, labels=labels)

    cv2.imwrite(
        f'yolo2/val/images/{counter:05X}.png', transformed_sample['image'])

    ann = ''
    for box, lbl in zip(transformed_sample['bboxes'], transformed_sample['labels']):
        ann += str(lbl[0]) + ' ' + ' '.join([str(b) for b in box])
        ann += '\n'

    Path(f'yolo2/val/labels/{counter:05X}.txt').write_text(ann)

    counter += 1


Val images:   0%|          | 0/10 [00:00<?, ?it/s]


KeyError: 'raw_backup/new/Нурик1092/Ainur2308/327010D01201909060002.tif'

In [ ]:
counter = 0
for idx, img_path in tqdm.tqdm(enumerate(balanced_train_images),
                               desc='Train images',
                               total=len(balanced_train_images)):
    img = loaded_images[img_path]
    # img = cv2.imread(img_path, 0)
    # img = cv2.resize(img, (int(img.shape[1] * 1024 / img.shape[0]), 1024))

    sel = train_annotations\
        .loc[train_annotations['filepath'] == img_path].copy()

    bboxes = sel[['x', 'y', 'width', 'height']
                 ].to_records(index=False).tolist()
    labels = sel[['label_id']].to_records(index=False).tolist()

    w = img.shape[1]
    h = img.shape[0]

    cut_lines = np.linspace(0, w, math.ceil(w / h)*2-1)[1:-2]
    cut_lines += np.random.normal(0, math.sqrt(h)/2, len(cut_lines))
    cut_lines = np.array([0, *cut_lines, w-h], dtype=np.int32).clip(0, w-h)

    for cut in cut_lines:
        cropper = alb.Compose([
            alb.Crop(cut, 0, cut+1024, 1024, always_apply=True)],
            bbox_params=bbox_params
        )
        cropped_img = cropper(image=img,
                              bboxes=bboxes,
                              labels=labels)
        transformed_sample = aug(**cropped_img)
        # display(transformed_sample)

        cv2.imwrite(
            f'yolo2/train/images/{counter:05X}.png', transformed_sample['image'])

        ann = ''
        for box, lbl in zip(transformed_sample['bboxes'], transformed_sample['labels']):
            ann += str(lbl[0]) + ' ' + ' '.join([str(b) for b in box])
            ann += '\n'

        Path(f'yolo2/train/labels/{counter:05X}.txt').write_text(ann)

        counter += 1

    break


In [ ]:
# for idx, img_loc in tqdm.tqdm(enumerate(balanced_train_images),
#                               total=len(balanced_train_images)):

#     image = Image.open(img_loc)
#     if image.mode == 'I;16':
#         image = image.convert('I').point(
#             [i/256 for i in range(2**16)], 'L')
#     else:
#         image = image.convert('L')

#     sel = train_annotations.loc[train_annotations['filepath'] == img_loc].copy(
#     )

#     sel['x'] = sel['x_min'] + sel['width'] // 2
#     sel['y'] = sel['y_min'] + sel['height'] // 2

#     sel['x'] /= image.width
#     sel['y'] /= image.height
#     sel['width'] /= image.width
#     sel['height'] /= image.height

#     bboxes = sel[['x', 'y', 'width', 'height']
#                  ].to_records(index=False).tolist()
#     labels = [selected_labels.index[selected_labels == lbl][0]
#               for lbl in sel['label'].tolist()]

#     sample = {
#         'image': np.asarray(image),
#         'bboxes': bboxes,
#         'labels': labels,
#     }

#     image.close()
#     try:

#         transformed_sample = aug(**sample)
#     except:
#         print(img_loc)
#         continue

#     image = Image.fromarray(transformed_sample['image'])
#     image.save(f'yolo/train/images/{idx:05X}.png')
#     image.close()

#     bboxes = transformed_sample['bboxes']
#     labels = transformed_sample['labels']

#     bboxes = '\n'.join([
#         f'{lbl} {box[0]} {box[1]} {box[2]} {box[3]}' for box, lbl in zip(bboxes, labels)]
#     )

#     Path(f'yolo/train/labels/{idx:05X}.txt').write_text(bboxes)


#     # bboxes = [(int((x-w/2)*500),
#     #            int((y-h/2)*500),
#     #            int((x+w/2)*500),
#     #            int((y+h/2)*500),) for x, y, w, h, in transformed_sample['bboxes']]

#     # wbboxes = tf.ToPILImage()(torchvision.utils.draw_bounding_boxes(
#     #     torch.from_numpy(transformed_sample['image']).expand(3, -1, -1),
#     #     boxes=torch.IntTensor(bboxes),
#     #     labels=[str(l) for l in transformed_sample['labels']],
#     #     fill=True,
#     #     colors=['red'] * len(bboxes),
#     # ))
    
#     # display(wbboxes)
